In [1]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Install required packages
!pip install opencv-python numpy tensorflow pillow

# Step 3: Import libraries
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import matplotlib.pyplot as plt
from IPython.display import display, Javascript, HTML
import ipywidgets as widgets
from datetime import datetime, time

# Step 4: Load and prepare dataset
def load_dataset(dataset_path):
    X_train, y_train = [], []
    X_test, y_test = [], []

    # Load training data
    train_path = os.path.join(dataset_path, 'train')
    for class_name in os.listdir(train_path):
        class_path = os.path.join(train_path, class_name)
        for img_file in os.listdir(class_path):
            img_path = os.path.join(class_path, img_file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (64, 64))
            X_train.append(img)
            y_train.append(class_name)

    # Load test data
    test_path = os.path.join(dataset_path, 'test')
    for class_name in os.listdir(test_path):
        class_path = os.path.join(test_path, class_name)
        for img_file in os.listdir(class_path):
            img_path = os.path.join(class_path, img_file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (64, 64))
            X_test.append(img)
            y_test.append(class_name)

    # Convert to numpy arrays
    X_train = np.array(X_train)
    X_test = np.array(X_test)

    # Encode labels
    le = LabelEncoder()
    y_train = le.fit_transform(y_train)
    y_test = le.transform(y_test)

    # One-hot encode
    num_classes = len(le.classes_)
    y_train = to_categorical(y_train, num_classes)
    y_test = to_categorical(y_test, num_classes)

    # Normalize and reshape
    X_train = X_train.astype('float32') / 255.0
    X_test = X_test.astype('float32') / 255.0
    X_train = np.expand_dims(X_train, axis=-1)
    X_test = np.expand_dims(X_test, axis=-1)

    return (X_train, y_train), (X_test, y_test), le.classes_

# Update this path to your dataset location in Google Drive
dataset_path = '/content/drive/MyDrive/Colab_Notebooks/isl_data_grey_split'
(X_train, y_train), (X_test, y_test), class_names = load_dataset(dataset_path)

print(f"Dataset loaded successfully with {len(class_names)} classes: {class_names}")
print(f"Training samples: {X_train.shape[0]}, Test samples: {X_test.shape[0]}")

# Step 5: Create and train the model
def create_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        MaxPooling2D(2,2),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Conv2D(128, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])
    return model

input_shape = X_train.shape[1:]
num_classes = len(class_names)
model = create_model(input_shape, num_classes)

history = model.fit(X_train, y_train,
                   validation_data=(X_test, y_test),
                   epochs=15,
                   batch_size=32)

# Step 6: Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"\nTest Accuracy: {test_acc*100:.2f}%")

# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Accuracy over epochs')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss over epochs')
plt.legend()
plt.show()

# Step 7: Save the model
model.save('/content/drive/MyDrive/Colab_Notebooks/sign_language_model.h5')
print("Model saved to Google Drive")

# Step 8: Real-time detection functions
def is_operational():
    now = datetime.now().time()
    start_time = time(18, 0)  # 6 PM
    end_time = time(22, 0)    # 10 PM
    return start_time <= now <= end_time

def predict_image(img_array):
    img = cv2.resize(img_array, (64, 64))
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=0)
    img = np.expand_dims(img, axis=-1)
    prediction = model.predict(img)
    return class_names[np.argmax(prediction)]

# Step 9: Create Colab interface
from IPython.display import display, clear_output
import ipywidgets as widgets

# Image upload function
def upload_image(change):
    if not is_operational():
        print("Service only available from 6 PM to 10 PM")
        return

    for file in uploader.value:
        content = file['content']
        with open('temp.jpg', 'wb') as f:
            f.write(content)

        img = cv2.imread('temp.jpg', cv2.IMREAD_GRAYSCALE)
        prediction = predict_image(img)

        # Display image and prediction
        display_img = Image.open('temp.jpg')
        plt.imshow(display_img, cmap='gray')
        plt.title(f"Predicted: {prediction}")
        plt.axis('off')
        plt.show()

# Camera capture function
def capture_image(button):
    if not is_operational():
        print("Service only available from 6 PM to 10 PM")
        return

    # JavaScript to access webcam
    js = Javascript('''
        async function capture() {
            const div = document.createElement('div');
            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});

            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();

            // Resize the output to fit the video element.
            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

            // Wait for Capture button to be clicked.
            await new Promise((resolve) => {
                const button = document.createElement('button');
                button.textContent = 'Capture';
                button.style.display = 'block';
                button.style.margin = '10px auto';
                div.appendChild(button);

                button.onclick = () => {
                    const canvas = document.createElement('canvas');
                    canvas.width = video.videoWidth;
                    canvas.height = video.videoHeight;
                    canvas.getContext('2d').drawImage(video, 0, 0);
                    stream.getVideoTracks()[0].stop();
                    div.remove();
                    resolve(canvas.toDataURL('image/jpeg'));
                };
            });
        }
        await capture();
        ''')

    # Display the JavaScript
    display(js)

    # Convert the image data to a numpy array
    def process_image(image_data):
        # Decode the base64 image data
        from base64 import b64decode
        import io
        image_bytes = b64decode(image_data.split(',')[1])
        image = Image.open(io.BytesIO(image_bytes))
        image.save('capture.jpg', 'JPEG')

        # Process and predict
        img = cv2.imread('capture.jpg', cv2.IMREAD_GRAYSCALE)
        prediction = predict_image(img)

        # Display results
        clear_output()
        plt.imshow(image, cmap='gray')
        plt.title(f"Predicted: {prediction}")
        plt.axis('off')
        plt.show()

    # Register callback
    js.on_done(process_image)

# Create widgets
uploader = widgets.FileUpload(accept='image/*', multiple=False)
upload_button = widgets.Button(description="Upload Image")
capture_button = widgets.Button(description="Capture from Camera")

upload_button.on_click(lambda b: upload_image(None))
capture_button.on_click(capture_image)

# Display the interface
print("## Sign Language Detection (6PM-10PM) ##")
display(uploader)
display(upload_button)
display(capture_button)

Mounted at /content/drive
Dataset loaded successfully with 15 classes: ['l' 'm' 'n' 'o' 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']
Training samples: 8430, Test samples: 0


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/15
264/264 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step - accuracy: 0.7208 - loss: 0.9047

ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(32, 1), dtype=float32). Expected shape (None, 64, 64, 1), but input has incompatible shape (32, 1)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(32, 1), dtype=float32)
  • training=False
  • mask=None